In [1]:
'''
Categorize conditional p-sites, universal p-sites, and random S/T according to the exposure
and compare the evolutionary rate in each exposure category. 
'''

import numpy as np
import pandas as pd
import pickle
from lib_import_tools import map_protein_id_to_locus_id
from pathlib import Path
from plot_tools import plot_consurf_exposure
from proteomic_tools import (retrieve_references_by_residue_type, 
							 get_phosphosites_given_perturbations,
                             retrieve_ConSurf_score, 
							 sample_random_sites, 
							 retrieve_references_by_order, 
							 calculate_exposure_consurf)

In [26]:
def default_list_dict():
    return defaultdict(list)

def default_str_dict():
    return defaultdict(str)

def parse_rsa_d(inPath, IDMappingDict, method='maximum'):
    # method can be average or maximum
    aaInfo_all, RSAInfo_all, deltaRSAInfo_all = pickle.load(open(inPath, 'rb'))
    RSAInfo_filtered = {}
    if method == "maximum":
        for protein, residues in RSAInfo_all.items():
            RSAInfo_filtered[protein] = {resnum: max(RSAs) for resnum, RSAs in residues.items()}
    elif method == "average":
        for protein, residues in RSAInfo_all.items():
            RSAInfo_filtered[protein] = {resnum: sum(RSAs) / len(RSAs) for resnum, RSAs in residues.items()}
    elif method == "minimum":
        for protein, residues in RSAInfo_all.items():
            RSAInfo_filtered[protein] = {resnum: min(RSAs) for resnum, RSAs in residues.items()}
    deltaRSAInfo_filtered = {}
    for protein, residues in deltaRSAInfo_all.items():
        deltaRSAInfo_filtered[protein] = {resnum: any(deltaRSA > 0 for deltaRSA in deltaRSAs) for resnum, deltaRSAs in residues.items()}
    aaInfo_all = {IDMappingDict[key]: value for key, value in aaInfo_all.items() if key in IDMappingDict}
    RSAInfo_filtered = {IDMappingDict[key]: value for key, value in RSAInfo_filtered.items() if key in IDMappingDict}
    deltaRSAInfo_filtered = {IDMappingDict[key]: value for key, value in deltaRSAInfo_filtered.items() if key in IDMappingDict}
    return aaInfo_all, RSAInfo_filtered, deltaRSAInfo_filtered

def calculate_exposed_consurf(exposed_residues_consurf_d: dict[str, float], 
                              sites_d: dict[str, set[str]],
                              consurf_d: dict[str, dict[int, float]], 
                              diso: dict[str, dict[int, float]],
                              aaInfo: dict[str, dict[int, str]], 
                              RSAInfo: dict[str, dict[int, float]], 
                              dRSAInfo: dict[str, dict[int, float]]) -> dict[str, float]:
    for key, sites in sites_d.items(): 
        if key == 'randomAll':
            sample_residues = 'ACDEFGHIKLMNPQRSTVWY'
        else:
            sample_residues = 'ST'
        interfacial_psite = []
        exposed_psite = []
        buried_psite = []
        psites_ST = retrieve_references_by_residue_type(sites, sample_residues)
        psites_dis = retrieve_references_by_order(psites_ST, diso, 'disordered')
        for psite in psites_dis: 
            systematic_name, site = psite.split('_')
            if systematic_name in ['YKL021C', 'YDR098C', 'YMR112C']:
                continue
            aa = site[0]
            position = int(site[1:])
            try: 
                if dRSAInfo[systematic_name][position] > 0:
                    interfacial_psite.append(psite)
                elif RSAInfo[systematic_name][position] > 0.25:
                    exposed_psite.append(psite)
                elif RSAInfo[systematic_name][position] <= 0.25:
                    buried_psite.append(psite)
            except KeyError:
                continue
        _, consurf_exposed = retrieve_ConSurf_score(exposed_psite, consurf_d)
        _, consurf_interfacial = retrieve_ConSurf_score(buried_psite, consurf_d)
        _, consurf_buried = retrieve_ConSurf_score(interfacial_psite, consurf_d)
        print(key, len(consurf_exposed), np.median(consurf_exposed), len(consurf_interfacial), np.median(consurf_interfacial), len(consurf_buried), np.median(consurf_buried))
        print(np.median([ele for ls in [consurf_exposed, consurf_interfacial, consurf_buried] for ele in ls]))
        exposed_residues_consurf_d[key] = np.median(consurf_exposed)
    return exposed_residues_consurf_d

In [16]:
def calculate_all_consurf(all_residues_consurf_d: dict[str, float], 
                          sites_d: dict[str, set[str]],
                          consurf_d: dict[str, dict[int, float]], 
                          diso: dict[str, dict[int, float]]) -> dict[str, float]:
    for key, sites in sites_d.items(): 
        if key == 'randomAll':
            sample_residues = 'ACDEFGHIKLMNPQRSTVWY'
        else:
            sample_residues = 'ST'
        psites_ST = retrieve_references_by_residue_type(sites, sample_residues)
        psites_dis = retrieve_references_by_order(psites_ST, diso, 'disordered')
        _, consurf_all = retrieve_ConSurf_score(psites_dis, consurf_d)
        all_residues_consurf_d[key] = np.median(consurf_all)
    return all_residues_consurf_d

In [3]:
method = 'maximum'

figFmt = 'jpg'

sample_residues = 'ST'

dataDir = Path('../../data')

extDir = dataDir / 'external'

procDir = dataDir / 'processed'

paperDir = procDir / 'paper'

# input files
IDMappingFile = extDir / 'YEAST_559292_idmapping.dat'
ultradeepPKL = paperDir / 'ultradeep_reference_phosphoproteome.pkl'
phosStresPKL = paperDir / 'quantitative_phosphosites.pkl'
consurfPKL = paperDir / 'consurf_all.pkl'
disoPKL = paperDir / 'diso_all.pkl'
sequencePKL = paperDir / 'Scer_seq.pkl'
sgdPKL = paperDir / 'SGD.pkl'
biogridPKL = paperDir / 'BioGRID.pkl'
lanz90PKL = paperDir / 'lanz90.pkl'
rsa_pkl = procDir / 'RSA_dicts.pkl'

# output files
Fig6 = paperDir / f'Figure 6 {method}.jpg'

IDMappingDict = map_protein_id_to_locus_id(IDMappingFile)
ultradeep = pickle.load(open(ultradeepPKL, 'rb'))
phosStres = pickle.load(open(phosStresPKL, 'rb'))
consurf = pickle.load(open(consurfPKL, 'rb'))
diso = pickle.load(open(disoPKL, 'rb'))
sequences = pickle.load(open(sequencePKL, 'rb'))
sgd = pickle.load(open(sgdPKL, 'rb'))
biogrid = pickle.load(open(biogridPKL, 'rb'))
lanz90 = pickle.load(open(lanz90PKL, 'rb'))
aaInfo, RSAInfo, dRSAInfo = parse_rsa_d(rsa_pkl, IDMappingDict, method=method)

In [4]:
phosStres = {key: retrieve_references_by_residue_type(references, sample_residues) for key, references in phosStres.items()}
cond_psites = get_phosphosites_given_perturbations(phosStres, list(range(1, 11)))
univ_psites = get_phosphosites_given_perturbations(phosStres, list(range(92, 102)))
all_psites = get_phosphosites_given_perturbations(phosStres, list(range(1, 102)))

exposed_consurf_d = {}

exclusions = ultradeep.union(sgd, biogrid) # All reported p-sites
randomST = sample_random_sites(ultradeep, ultradeep, sequences, sample_residues)
randomST_all = sample_random_sites(ultradeep, exclusions, sequences, sample_residues)
randomAll = sample_random_sites(ultradeep, exclusions, sequences, 'ACDEFGHIKLMNPQRSTVWY')

In [6]:
len(randomAll)

2165818

In [27]:
sites_d = {
    'cond': cond_psites, 
    'univ': univ_psites, 
    'all': all_psites, 
    'biogrid': biogrid,
    'sgd': sgd, 
    'lanz90': lanz90, 
    'randomST': randomST,
    'randomST_all': randomST_all,
    'randomAll': randomAll
}

exposed_consurf_d = calculate_exposed_consurf(exposed_consurf_d, 
                                              sites_d,
                                              consurf, 
                                              diso, 
                                              aaInfo, 
                                              RSAInfo, 
                                              dRSAInfo)

cond 1159 0.4513 15 -0.2239 62 -0.007850000000000001
0.41755
univ 904 0.1683 10 0.6661 76 -0.07055
0.1411
all 3913 0.3151 45 0.1174 244 -0.037000000000000005
0.2948
biogrid 2310 0.25655 28 -0.023799999999999995 198 0.1285
0.22244999999999998
sgd 4663 0.3394 58 -0.2468 346 0.01585
0.3102
lanz90 3146 0.33485 39 -0.4422 241 0.0809
0.3029
randomST 12871 0.3352 356 -0.35845 1226 0.07744999999999999
0.2945
randomST_all 11147 0.3206 331 -0.3599 1077 0.0649
0.2772
randomAll 92727 0.2266 2438 -0.42725 9972 -0.1445
0.1673


In [ ]:
all_consurf_d = {}
all_consurf_d = calculate_all_consurf(all_consurf_d,
                                      sites_d,
                                      consurf,
                                      diso)

for key, consurf_median in exposed_consurf_d.items():
    print(f'{key}:\t{consurf_median}')

cond 1159
univ 904
all 3913
biogrid 2310
sgd 4663
lanz90 3146
randomST 12871
randomST_all 11147
randomAll 92727
cond:	0.4513
univ:	0.1683
all:	0.3151
biogrid:	0.25655
sgd:	0.3394
lanz90:	0.33485
randomST:	0.3352
randomST_all:	0.3206
randomAll:	0.2266


In [18]:
print(all_consurf_d)

{'cond': np.float64(0.3463), 'univ': np.float64(0.1163), 'all': np.float64(0.25734999999999997), 'biogrid': np.float64(0.1991), 'sgd': np.float64(0.2736), 'lanz90': np.float64(0.2659), 'randomST': np.float64(0.2958), 'randomST_all': np.float64(0.29075), 'randomAll': np.float64(0.2257)}
